## Imports

In [92]:
# Std Libs
import requests
import io
from datetime import datetime

# Third party libs
import pandas as pd
import pymongo
from pymongo import MongoClient

#To avoid deprecation warnings
import warnings
warnings.filterwarnings('ignore')

## Mongo db is running inside a container on local host

In [3]:
client = MongoClient('mongodb://localhost:27017/',
username='root',
password='MongoDB2020')

## Mong express from the compose is available at http://penguin.linux.test:8081

## DB Operations

### Create a new db and collection.

In [8]:
db = client['database']
coll = db['covid_uk']

In [9]:
client.list_database_names()

['admin', 'config', 'database', 'local']

In [76]:
db.list_collection_names()

['covid_uk']

### Loading data from Pandas to Mongo

#### Get in some data for testing.

In [14]:
def read_in_data():
    datastr = requests.get(
        'https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv', allow_redirects=True).text
    data_file = io.StringIO(datastr)
    d = pd.read_csv(data_file, parse_dates=['Specimen date'])
    return d

In [15]:
d = read_in_data()

In [17]:
d.shape

(8400, 10)

In [18]:
d.head()

,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,England,E92000001,Nation,2020-05-01,95.0,0.0,95.0,122842,120614.0,2228.0
1,South West,E12000009,Region,2020-05-01,5.0,NaN,NaN,6405,NaN,NaN
2,South East,E12000008,Region,2020-05-01,11.0,NaN,NaN,17324,NaN,NaN
3,London,E12000007,Region,2020-05-01,3.0,NaN,NaN,24700,NaN,NaN
4,East of England,E12000006,Region,2020-05-01,7.0,NaN,NaN,10785,NaN,NaN


In [19]:
df_dict = d.to_dict(orient='records')

In [20]:
type(df_dict)

list

In [21]:
df_dict[0]

{'Area name': 'England',
 'Area code': 'E92000001',
 'Area type': 'Nation',
 'Specimen date': Timestamp('2020-05-01 00:00:00'),
 'Daily lab-confirmed cases': 95.0,
 'Previously reported daily cases': 0.0,
 'Change in daily cases': 95.0,
 'Cumulative lab-confirmed cases': 122842,
 'Previously reported cumulative cases': 120614.0,
 'Change in cumulative cases': 2228.0}

In [21]:
coll.insert_many(df_dict)

Confirm Insertions - So we have all records in the collection.

In [93]:
coll.find().count()

8400

### Various DB read queries - Load output to Pandas DF

Read in the whole collection

In [94]:
coll.find().count()

8400

In [89]:
pd.DataFrame(coll.find()).shape

(8400, 11)

In [91]:
pd.DataFrame(coll.find()).head(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abae8f,England,E92000001,Nation,2020-05-01,95.0,0.0,95.0,122842,120614.0,2228.0
1,5eadc3722a6eadec96abae90,South West,E12000009,Region,2020-05-01,5.0,NaN,NaN,6405,NaN,NaN


## Single filter; Area Name = Islington

In [95]:
coll.find({'Area name': 'Islington'}).count()

54

In [42]:
coll.find_one({'Area name': 'Islington'})

{'_id': ObjectId('5eadc3722a6eadec96abaec1'),
 'Area name': 'Islington',
 'Area code': 'E09000019',
 'Area type': 'Upper tier local authority',
 'Specimen date': datetime.datetime(2020, 5, 1, 0, 0),
 'Daily lab-confirmed cases': 0.0,
 'Previously reported daily cases': nan,
 'Change in daily cases': nan,
 'Cumulative lab-confirmed cases': 414,
 'Previously reported cumulative cases': nan,
 'Change in cumulative cases': nan}

In [43]:
type(_)

dict

In [37]:
pd.DataFrame(coll.find({'Area name': 'Islington'})).head(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abaec1,Islington,E09000019,Upper tier local authority,2020-05-01,0.0,NaN,NaN,414,NaN,NaN
1,5eadc3722a6eadec96abaf5b,Islington,E09000019,Upper tier local authority,2020-04-30,1.0,NaN,NaN,414,NaN,NaN


AND filter

In [96]:
coll.find({"Area name":"Hounslow","Area type":"Upper tier local authority"}).count()

56

In [38]:
pd.DataFrame(coll.find({"Area name":"Hounslow","Area type":"Upper tier local authority"})).head(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abaec2,Hounslow,E09000018,Upper tier local authority,2020-05-01,0.0,NaN,NaN,653,NaN,NaN
1,5eadc3722a6eadec96abaf5c,Hounslow,E09000018,Upper tier local authority,2020-04-30,4.0,NaN,NaN,653,NaN,NaN


OR Filter

In [97]:
coll.find({"$or":[{"Area name":"Hounslow"},{"Area name":"Islington"}]}).count()

110

In [46]:
pd.DataFrame(coll.find({"$or":[{"Area name":"Hounslow"},{"Area name":"Islington"}]})).sample(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
37,5eadc3722a6eadec96abba49,Hounslow,E09000018,Upper tier local authority,2020-04-12,10.0,NaN,NaN,503,NaN,NaN
19,5eadc3722a6eadec96abb4bd,Islington,E09000019,Upper tier local authority,2020-04-21,2.0,NaN,NaN,391,NaN,NaN


IN Filter

In [98]:
coll.find({"Area name":{"$in":["Hounslow","Islington"]}}).count()

110

In [100]:
pd.DataFrame(coll.find({"Area name":{"$in":["Hounslow","Islington"]}})).sample(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
62,5eadc3722a6eadec96abc251,Islington,E09000019,Upper tier local authority,2020-03-30,18.0,NaN,NaN,194,NaN,NaN
85,5eadc3722a6eadec96abc8f4,Hounslow,E09000018,Upper tier local authority,2020-03-19,13.0,NaN,NaN,67,NaN,NaN


NOT IN Filter

In [101]:
coll.find({"Area name":{"$nin":["Upper tier local authority	","Islington"]}}).count() 

8346

In [54]:
pd.DataFrame(coll.find({"Area name":{"$nin":["Hounslow","Islington"]}})).head(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abae8f,England,E92000001,Nation,2020-05-01,95.0,0.0,95.0,122842,120614.0,2228.0
1,5eadc3722a6eadec96abae90,South West,E12000009,Region,2020-05-01,5.0,NaN,NaN,6405,NaN,NaN


Greater than - While also using date

In [102]:
coll.find({'Specimen date':{"$gt":datetime(2020,4,30)}}).count()

160

In [73]:
pd.DataFrame(coll.find({'Specimen date':{"$gt":datetime(2020,4,30)}})).head(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abae8f,England,E92000001,Nation,2020-05-01,95.0,0.0,95.0,122842,120614.0,2228.0
1,5eadc3722a6eadec96abae90,South West,E12000009,Region,2020-05-01,5.0,NaN,NaN,6405,NaN,NaN


Between - Using dates

In [103]:
coll.find({"Specimen date": {"$gte": datetime(2020,4,30), "$lt": datetime(2020,5,1)}}).count()

125

In [75]:
pd.DataFrame(coll.find({"Specimen date": {"$gte": datetime(2020,4,30), "$lt": datetime(2020,5,1)}})).head(2)

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abaf2f,England,E92000001,Nation,2020-04-30,707.0,85.0,622.0,122747,120614.0,2133.0
1,5eadc3722a6eadec96abaf30,South West,E12000009,Region,2020-04-30,46.0,NaN,NaN,6400,NaN,NaN


Regex

In [58]:
import re
regex = re.compile(r'Engla*')
rstats = coll.find_one({"Area name":regex})
rstats

{'_id': ObjectId('5eadc3722a6eadec96abae8f'),
 'Area name': 'England',
 'Area code': 'E92000001',
 'Area type': 'Nation',
 'Specimen date': datetime.datetime(2020, 5, 1, 0, 0),
 'Daily lab-confirmed cases': 95.0,
 'Previously reported daily cases': 0.0,
 'Change in daily cases': 95.0,
 'Cumulative lab-confirmed cases': 122842,
 'Previously reported cumulative cases': 120614.0,
 'Change in cumulative cases': 2228.0}

Filter, Sort and Limit - Five worse days in terms of cases for a borough

In [65]:
pd.DataFrame(coll.find({"Area name":"Hounslow"}).sort([("Daily lab-confirmed cases", pymongo.DESCENDING)]).limit(5))

,_id,Area name,Area code,Area type,Specimen date,Daily lab-confirmed cases,Previously reported daily cases,Change in daily cases,Cumulative lab-confirmed cases,Previously reported cumulative cases,Change in cumulative cases
0,5eadc3722a6eadec96abc077,Hounslow,E09000018,Upper tier local authority,2020-04-02,33.0,NaN,NaN,321,NaN,NaN
1,5eadc3722a6eadec96abc115,Hounslow,E09000018,Upper tier local authority,2020-04-01,32.0,NaN,NaN,288,NaN,NaN
2,5eadc3722a6eadec96abc42c,Hounslow,E09000018,Upper tier local authority,2020-03-27,27.0,NaN,NaN,187,NaN,NaN
3,5eadc3722a6eadec96abbc25,Hounslow,E09000018,Upper tier local authority,2020-04-09,24.0,NaN,NaN,468,NaN,NaN
4,5eadc3722a6eadec96abbfd9,Hounslow,E09000018,Upper tier local authority,2020-04-03,24.0,NaN,NaN,345,NaN,NaN


### Updating Documents in Mongo

In [136]:
db.drop_collection('counter')

{'nIndexesWas': 1, 'ns': 'database.counter', 'ok': 1.0}

In [137]:
counter_coll = db['counter']

In [138]:
counter_coll.insert_many([{'process_id':'ABC','run':10},{'process_id':'DEF','run':54},{'process_id':'DEF','run':77},{'process_id':'GHI','run':100},{'process_id':'JKL','run':392}])

In [139]:
[x for x in counter_coll.find()]

[{'_id': ObjectId('5eae90872defff6ac9d117ed'), 'process_id': 'ABC', 'run': 10},
 {'_id': ObjectId('5eae90872defff6ac9d117ee'), 'process_id': 'DEF', 'run': 54},
 {'_id': ObjectId('5eae90872defff6ac9d117ef'), 'process_id': 'DEF', 'run': 77},
 {'_id': ObjectId('5eae90872defff6ac9d117f0'),
  'process_id': 'GHI',
  'run': 100},
 {'_id': ObjectId('5eae90872defff6ac9d117f1'),
  'process_id': 'JKL',
  'run': 392}]

In my use case, I will have dict with updated values for key, we need to update them in mongo.

In [140]:
new_d = dict({'ABC':23,'DEF':34,'JKL':305})

In [141]:
new_d

{'ABC': 23, 'DEF': 34, 'JKL': 305}

Not sure if this is best way of doing it, but should do for now.

In [144]:
for k,v in new_d.items():
    counter_coll.update({'process_id':k},{'run':v})

In [145]:
[x for x in counter_coll.find()]

[{'_id': ObjectId('5eae90872defff6ac9d117ed'), 'run': 23},
 {'_id': ObjectId('5eae90872defff6ac9d117ee'), 'run': 34},
 {'_id': ObjectId('5eae90872defff6ac9d117ef'), 'process_id': 'DEF', 'run': 77},
 {'_id': ObjectId('5eae90872defff6ac9d117f0'),
  'process_id': 'GHI',
  'run': 100},
 {'_id': ObjectId('5eae90872defff6ac9d117f1'), 'run': 305}]